1. Import all required liberaries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

2. Read the Data

In [4]:
movie_list = pd.read_csv('tmdb_5000_movies.csv')
credits_list = pd.read_csv('tmdb_5000_credits.csv') 

3. Merger both the DataSet

In [7]:
movie_list = movie_list.merge(credits_list,on='title')

- Filter Dataset with the required columns

In [8]:
# genres
# id
# keywords
# title
# overview
# cast
# crew

movie_list = movie_list[['movie_id','title','overview','genres','keywords','cast','crew']]
movie_list.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


- Check if any missing data and clear

In [9]:
movie_list.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movie_list.dropna(inplace=True)

- Check for duplicate data

In [11]:
movie_list.duplicated().sum()

0

4. Data-Preprocessing:
- Column Genres

In [12]:
movie_list.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
import ast
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [14]:
movie_list['genres'] = movie_list['genres'].apply(convert)

- Column Keywords

In [15]:
movie_list['keywords'] = movie_list['keywords'].apply(convert)

In [16]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

- Column Cast

In [17]:
def convert3(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
    if counter!=3:
      L.append(i['name'])
      counter +=1
    else:
      break
  return L

In [18]:
movie_list['cast'] = movie_list['cast'].apply(convert)

In [19]:
movie_list['cast'] = movie_list['cast'].apply(lambda x:x[0:3])

- Column Crew(Director)

In [20]:
def fetch_Director(obj):
  L = []
  counter = 0
  for i in ast.literal_eval(obj):
      if i['job'] == 'Director':
        L.append(i['name'])
        break
  return L

In [21]:
movie_list['crew'] = movie_list['crew'].apply(fetch_Director)

- Column Overview

In [22]:
#movie_list['overview'] = movie_list['overview'].apply(lambda x:x.split())
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [23]:
movie_list['cast'] = movie_list['cast'].apply(collapse)
movie_list['crew'] = movie_list['crew'].apply(collapse)
movie_list['genres'] = movie_list['genres'].apply(collapse)
movie_list['keywords'] = movie_list['keywords'].apply(collapse)

In [24]:
movie_list['overview'] = movie_list['overview'].apply(lambda x:x.split())
movie_list['tags'] = movie_list['overview'] + movie_list['genres'] + movie_list['keywords'] + movie_list['cast'] + movie_list['crew']
new_df = movie_list.drop(columns=['overview','genres','keywords','cast','crew'])
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [25]:
#movie_list['genres'] = movie_list['genres'].apply(lambda x:[i.replace(" ","") for i in x])
#movie_list['keywords'] = movie_list['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
#movie_list['cast'] = movie_list['cast'].apply(lambda x:[i.replace(" ","") for i in x])
#movie_list['crew'] = movie_list['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
movie_list.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


- Creating a Column Tags

In [27]:
movie_list['tags'] = movie_list['overview'] + movie_list['genres'] + movie_list['keywords'] + movie_list['cast'] + movie_list['crew']

In [28]:
movie_list.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [29]:
new_df = movie_list[['movie_id','title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\smitt\AppData\Local\Temp\ipykernel_13540\1626043869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [30]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [31]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\smitt\AppData\Local\Temp\ipykernel_13540\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


5. Vectorization

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words ='english')

In [33]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [37]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
cv.get_feature_names_out

<bound method CountVectorizer.get_feature_names_out of CountVectorizer(max_features=5000, stop_words='english')>

6. Stemming

In [272]:
#!pip install nltk

In [273]:
#import nltk
#from nltk.stem.porter import PorterStemmer
#port = PorterStemmer()

In [274]:
#def stemming(text):
#  x=[]
#  for i in text.split():
#    x.append(port.stem(i))
#    return " ".join(x)

In [275]:
#new_df1 = new_df['tags'].apply(stemming)
#new_df['title'] = new_df['title'].str.lower()
#new_df['title'] = new_df['title'].str.replace(" ", "")

7. Finding Similatity
- Finding Cosine distance between each vectors

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity

array([[1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
        0.        ],
       [0.08964215, 1.        , 0.0625    , ..., 0.02635231, 0.        ,
        0.        ],
       [0.05976143, 0.0625    , 1.        , ..., 0.02635231, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02635231, ..., 1.        , 0.0745356 ,
        0.04836508],
       [0.02817181, 0.        , 0.        , ..., 0.0745356 , 1.        ,
        0.05407381],
       [0.        , 0.        , 0.        , ..., 0.04836508, 0.05407381,
        1.        ]])

In [278]:
def recommender(movie):

  movie_index = new_df[new_df['title'] == movie].index[0]
  distance = sorted(list(enumerate(similarity[movie_index])),reverse=True,key = lambda x:x[1])[1:15]
  #sort = sorted(list(enumerate(similarity[movie_index])),reverse=True,key = lambda x:x[1])[1:6]

  for i in distance[1:15]:
    print(new_df.iloc[i[0]].title)

In [280]:
recommender('Avengers: Age of Ultron')

Iron Man 3
The Avengers
Captain America: Civil War
Iron Man 2
Silver Medalist
Thor: The Dark World
Thor
Superman II
The Wolverine
Guardians of the Galaxy


In [281]:
import pickle

In [282]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))